In [ ]:
!pip install transformers datasets torch


In [ ]:
import pandas as pd
dataset = pd.read_csv('IMDB Dataset.csv')
print(dataset.columns)

Index(['review', 'sentiment'], dtype='object')


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
model = AutoModelForSequenceClassification.from_pretrained("NousResearch/Llama-2-7b-chat-hf", num_labels=2)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at NousResearch/Llama-2-7b-chat-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install datasets


In [ ]:
from datasets import Dataset

# Assuming 'dataset' is your pandas DataFrame
dataset = Dataset.from_pandas(dataset)



In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['review'], padding='max_length', truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)



Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
from datasets import DatasetDict

# Split the dataset into training and test sets (25% for testing)
train_test_split = tokenized_datasets.train_test_split(test_size=0.25)
dataset = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})


In [ ]:
!pip install accelerate -U


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    no_cuda=True,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1281: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
